<a href="https://colab.research.google.com/github/sachinbluechip/RAG-Q-A-with-Gemini-Pro-Google-/blob/main/RAG_Q%26A_with_Gemini_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.8/379.8 kB 24.2 MB/s eta 0

In [2]:
from IPython.display import display
from IPython.display import Markdown
import textwrap


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
import google.generativeai as genai
from google.colab import userdata

In [4]:
import os
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
model = genai.GenerativeModel(model_name = "gemini-pro")
model


genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [6]:
response = model.generate_content("What are the usecases of LLMs?")

In [7]:

to_markdown(response.text)

> **Business and Finance:**
> 
> * **Customer service:** Providing personalized and automated responses to customer inquiries.
> * **Content generation:** Creating high-quality content for marketing, social media, and advertising.
> * **Market research:** Analyzing customer feedback and industry trends to identify opportunities.
> * **Financial forecasting:** Predicting revenue, expenses, and investment returns.
> * **Data analysis:** Extracting insights from large datasets to inform decision-making.
> 
> **Education and Research:**
> 
> * **Personalized learning:** Tailoring educational content and experiences to individual students' needs.
> * **Academic writing assistance:** Generating abstracts, summaries, and research proposals.
> * **Answering questions:** Providing quick and accurate answers to research and academic inquiries.
> * **Language learning:** Enhancing vocabulary, grammar, and pronunciation.
> * **Scientific discovery:** Assisting researchers in exploring hypotheses and generating new knowledge.
> 
> **Healthcare:**
> 
> * **Patient diagnosis:** Analyzing medical data to identify diseases and provide treatment recommendations.
> * **Drug discovery:** Accelerating the development of new drugs and treatments.
> * **Personalized medicine:** Tailoring treatments to individual patients based on their genetic and health history.
> * **Clinical research:** Automating data collection, analysis, and reporting.
> * **Patient education:** Providing easy-to-understand information about medical conditions and treatments.
> 
> **Entertainment and Media:**
> 
> * **Creative writing:** Generating ideas, writing scripts, and creating fictional characters.
> * **Music composition:** Generating melodies, lyrics, and arrangements.
> * **Video editing:** Automatically summarizing and enhancing video content.
> * **Gaming:** Creating realistic and immersive game worlds and characters.
> * **Art generation:** Generating visual content, including images, paintings, and sculptures.
> 
> **Other Industries:**
> 
> * **Government:** Automating administrative tasks, analyzing public sentiment, and informing policy decisions.
> * **Legal:** Performing legal research, summarizing legal documents, and automating certain tasks.
> * **Retail:** Providing personalized product recommendations, automating customer support, and optimizing inventory management.
> * **Travel:** Generating travel itineraries, suggesting attractions, and booking accommodations.
> * **Transportation:** Optimizing vehicle routing, predicting traffic patterns, and assisting with navigation.

## Chat with Documents using RAG (Retreival Augment Generation)

In [8]:

!sudo apt -y -qq install tesseract-ocr libtesseract-dev

!sudo apt-get -y -qq install poppler-utils libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

!pip install langchain

The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr
  tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 6.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 123598 files and directories

In [9]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00


In [10]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA



warnings.filterwarnings("ignore")
# restart python kernal if issues with langchain import.

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [12]:
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY,
                             temperature=0.2,convert_system_message_to_human=True)

In [13]:
#Extract text from the PDF
pdf_loader = PyPDFLoader("/content/drive/MyDrive/RAG_documents/attention.pdf")
pages = pdf_loader.load_and_split()
print(pages[3].page_content)

Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two
sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-
wise fully connected feed-forward network. We employ a residual connection [ 11] around each of
the two sub-layers, followed by layer normalization [ 1]. That is, the output of each sub-layer is
LayerNorm( x+ Sublayer( x)), where Sublayer( x)is the function implemented by the sub-layer
itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding
layers, produce outputs of dimension dmodel = 512 .
Decoder: The decoder is also composed of a stack of N= 6identical layers.

In [14]:
len(pages)

16

In [15]:
#RAG Pipeline: Embedding + Gemini (LLM)

from langchain_google_genai import GoogleGenerativeAIEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)


embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)


vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})


qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True

)

In [16]:
question = "Describe the Multi-head attention layer in detail?"
result = qa_chain({"query": question})
result["result"]

'Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.\n\nMultiHead( Q, K, V ) = Concat(head 1, ...,head h)WO\nwhere head i= Attention( QWQ\ni, KWK\ni, V WV\ni)\n\nWhere the projections are parameter matrices WQ\ni∈Rdmodel×dk,WK\ni∈Rdmodel×dk,WV\ni∈Rdmodel×dv\nandWO∈Rhdv×dmodel.\n\nIn this work we employ h= 8 parallel attention layers, or heads. For each of these we use\ndk=dv=dmodel/h= 64 . Due to the reduced dimension of each head, the total computational cost\nis similar to that of single-head attention with full dimensionality.'

In [17]:

Markdown(result["result"])

Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.

MultiHead( Q, K, V ) = Concat(head 1, ...,head h)WO
where head i= Attention( QWQ
i, KWK
i, V WV
i)

Where the projections are parameter matrices WQ
i∈Rdmodel×dk,WK
i∈Rdmodel×dk,WV
i∈Rdmodel×dv
andWO∈Rhdv×dmodel.

In this work we employ h= 8 parallel attention layers, or heads. For each of these we use
dk=dv=dmodel/h= 64 . Due to the reduced dimension of each head, the total computational cost
is similar to that of single-head attention with full dimensionality.

In [18]:

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [19]:

question = "Describe the Multi-head attention layer in detail?"
result = qa_chain({"query": question})
result["result"]

'Multi-Head Attention consists of several attention layers running in parallel. Each attention layer is responsible for attending to a different subspace of the input. The outputs of the attention layers are then concatenated and projected to produce the final output.\nThe Multi-Head Attention layer is defined as follows:\n```\nMultiHead( Q, K, V ) = Concat(head 1, ...,head h)WO\nwhere head i= Attention( QWQ\ni, KWK\ni, V WV\ni)\n```\nWhere the projections are parameter matrices WQ\ni∈Rdmodel×dk,WK\ni∈Rdmodel×dk,WV\ni∈Rdmodel×dv\nandWO∈Rhdv×dmodel.\nIn this work we employ h= 8 parallel attention layers, or heads. For each of these we use\ndk=dv=dmodel/h= 64 . Due to the reduced dimension of each head, the total computational cost\nis similar to that of single-head attention with full dimensionality.\nMulti-head attention allows the model to jointly attend to information from different representation\nsubspaces at different positions. With a single attention head, averaging inhibits thi

In [20]:
Markdown(result["result"])

Multi-Head Attention consists of several attention layers running in parallel. Each attention layer is responsible for attending to a different subspace of the input. The outputs of the attention layers are then concatenated and projected to produce the final output.
The Multi-Head Attention layer is defined as follows:
```
MultiHead( Q, K, V ) = Concat(head 1, ...,head h)WO
where head i= Attention( QWQ
i, KWK
i, V WV
i)
```
Where the projections are parameter matrices WQ
i∈Rdmodel×dk,WK
i∈Rdmodel×dk,WV
i∈Rdmodel×dv
andWO∈Rhdv×dmodel.
In this work we employ h= 8 parallel attention layers, or heads. For each of these we use
dk=dv=dmodel/h= 64 . Due to the reduced dimension of each head, the total computational cost
is similar to that of single-head attention with full dimensionality.
Multi-head attention allows the model to jointly attend to information from different representation
subspaces at different positions. With a single attention head, averaging inhibits this.
Thanks for asking!

In [21]:
question = "Describe Random forest?"
result = qa_chain({"query": question})
Markdown(result["result"])

Random forest is a supervised learning algorithm that builds an ensemble of decision trees and outputs the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees.